# SQL and Data Viz

1. Identify the best month in terms of loan issuance. What was the quantity and amount lent in each month?
2. Which batch had the best overall adherence?
3. Do different interest rates lead to different loan outcomes in terms of default rate?
4. Rank the best 10 and 10 worst clients. Explain your methodology for constructing this ranking.
5. What is the default rate by month and batch?
6. Assess the profitability of this operation. Provide an analysis of the operation's timeline.

> adherence: clients that got loans\
> season: loan issuing month\
> default rate: defaulted/issued loans

## Importing Libraries and Establishing Database Connection

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Function to execute SQL queries and return results as a pandas DataFrame
def execute_query(query):
    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}")
    
    # Execute the query and return the result as a DataFrame
    with engine.connect() as connection:
        df = pd.read_sql_query(query, connection)
    return df

# Exploratory Data Analysis

In [4]:
query_five_first_on_loans = """
SELECT *
FROM loans
LIMIT 5;
"""

execute_query(query_five_first_on_loans)

,user_id,loan_id,created_at,due_at,paid_at,status,loan_amount,tax,due_amount,amount_paid
0,46937,1,2020-01-06 08:58:24,2020-04-05 08:58:24,2020-02-21 08:58:24,paid,16638.0,186.01,18071.86,18071.86
1,29211,2,2020-01-07 05:12:59,2020-04-06 05:12:59,2020-03-09 05:12:59,paid,1886.0,21.09,2331.44,2331.44
2,62030,3,2020-01-12 02:06:18,2020-04-11 02:06:18,NaT,default,39802.0,444.99,42237.09,4147.27
3,14500,4,2020-01-14 18:09:12,2020-04-13 18:09:12,2020-01-28 18:09:12,paid,5114.0,57.17,5554.72,5554.72
4,73480,5,2020-01-15 17:28:24,2020-04-14 17:28:24,2020-03-14 17:28:24,paid,22153.0,247.67,27385.10,27385.10


In [5]:
query_five_first_on_clients = """
SELECT *
FROM clients
LIMIT 5;
"""

execute_query(query_five_first_on_clients)

,user_id,created_at,status,batch,credit_limit,interest_rate,denied_reason,denied_at
0,1,2023-09-18 16:05:36,approved,1,47500,30,None,NaT
1,2,2020-07-05 07:00:37,denied,1,59750,20,money_loundry,2023-07-29 02:48:33
2,3,2023-07-25 03:39:55,approved,1,73000,30,None,NaT
3,4,2022-07-01 01:28:58,approved,1,14250,20,None,NaT
4,5,2023-06-23 20:17:40,approved,1,23750,20,None,NaT


In [6]:
# distinct values of the denied_reason column in the clients table
query_distinct_denied_reason = """
SELECT DISTINCT denied_reason
FROM clients;
"""

execute_query(query_distinct_denied_reason)

,denied_reason
0,money_loundry
1,defaulter
2,acccount_take_over
3,None
4,criminal_profile


In [7]:
#distinct values and counts of the denied_reason column in the clients table
query_distinct_denied_reason_and_counts = """
SELECT denied_reason, COUNT(denied_reason) AS count
FROM clients
GROUP BY denied_reason;
"""

execute_query(query_distinct_denied_reason_and_counts)

,denied_reason,count
0,money_loundry,1986
1,defaulter,12341
2,acccount_take_over,1980
3,None,0
4,criminal_profile,2034


In [8]:
#  distinct values and counts status from clients table
query_distinct_status = """
SELECT status, COUNT(status) AS count
FROM clients
GROUP BY status;
"""

execute_query(query_distinct_status)

,status,count
0,denied,18341
1,approved,71659


In [9]:
#how many loans were disbursed to each client
query_loans_disbursed_to_clients = """
SELECT COUNT(*)
FROM clients;
"""

execute_query(query_loans_disbursed_to_clients)

,count
0,90000


In [10]:
#how many loans
query_how_many_loans = """
SELECT COUNT(*) as count_loans
FROM loans;
"""

execute_query(query_how_many_loans)

,count_loans
0,150708


In [11]:
#how many loans were disbursed to each client
query_loans_disbursed_to_clients = """
SELECT user_id, count_loans
FROM (
    SELECT user_id, COUNT(*) as count_loans
    FROM loans
    GROUP BY user_id
) AS subquery
WHERE count_loans > 1
ORDER BY count_loans DESC;
"""

execute_query(query_loans_disbursed_to_clients)

,user_id,count_loans
0,62334,8
1,9912,8
2,14050,8
3,37345,8
4,2559,8
...,...,...
45052,45860,2
45053,1014,2
45054,79163,2
45055,7004,2


So we have clients that have more than one loan. We have to be careful with this, as we can't just sum the amount lent and the quantity of loans. We have to group by client and then by month.

In [12]:
# distinct values of the interest_rate column in the clients table
query_distinct_interest_rate = """
SELECT DISTINCT interest_rate
FROM clients;
"""

execute_query(query_distinct_interest_rate)

,interest_rate
0,30
1,90
2,70
3,20


# Answering the Questions

## 1. Identify the best month in terms of loan issuance. What was the quantity and amount lent in each month?

In [13]:
query_best_month = '''
SELECT 
    DATE_TRUNC('month', created_at) AS month,
    COUNT(loan_id) AS total_quantity,
    SUM(loan_amount) AS total_amount
FROM 
    loans
GROUP BY 
    DATE_TRUNC('month', created_at)
ORDER BY 
    total_amount DESC
LIMIT 1;
'''

In [14]:
execute_query(query_best_month)

,month,total_quantity,total_amount
0,2023-12-01,17351,442464966.0



The analysis indicates that December 2023 had the highest loan issuance, with a total of 17,351 loans issued and a total amount lent of $442,464,966.00. This information provides insights into the peak activity of loan issuance, which can be further analyzed to understand potential factors contributing to the increased demand for loans during that month.

In [15]:
#  What was the quantity and amount lent in each month?
query_monthly = '''
SELECT 
    DATE_TRUNC('month', created_at) AS month,
    COUNT(loan_id) AS total_quantity,
    SUM(loan_amount) AS total_amount
FROM
    loans
GROUP BY    
    DATE_TRUNC('month', created_at)
ORDER BY
    month;
'''

In [16]:
execute_query(query_monthly)

,month,total_quantity,total_amount
0,2020-01-01,16,348731.0
1,2020-02-01,59,1723978.0
2,2020-03-01,107,2460062.0
3,2020-04-01,145,3465180.0
4,2020-05-01,161,4323270.0
5,2020-06-01,224,5918356.0
6,2020-07-01,274,7086345.0
7,2020-08-01,314,7998350.0
8,2020-09-01,343,8852936.0
9,2020-10-01,464,11477276.0


## 2. Which batch had the best overall adherence?

In [17]:
# Batch with the best overall Adherence
query_best_adherence = '''
SELECT 
    c.batch AS batch_id,
    COUNT(l.user_id) AS total_loans,
    SUM(CASE WHEN l.status = 'paid' THEN 1 ELSE 0 END) AS paid_loans,
    SUM(CASE WHEN l.status = 'paid' THEN 1.0 ELSE 0 END) / COUNT(l.user_id) AS adherence
FROM 
    loans l
JOIN 
    clients c ON l.user_id = c.user_id
GROUP BY 
    c.batch
ORDER BY 
    adherence DESC;
'''

execute_query(query_best_adherence)

,batch_id,total_loans,paid_loans,adherence
0,2,37415,22558,0.602913
1,3,8958,5350,0.597232
2,1,98364,58248,0.592168
3,4,5971,3439,0.575950


the result indicates that batch number 2 had the highest proportion of clients who successfully repaid their loans compared to the other batches, with an adherence rate of approximately 60.29%. This suggests that clients in batch 2 demonstrated better adherence to loan repayment obligations compared to clients in other batches.

## 3. Do different interest rates lead to different loan outcomes in terms of default rate?

In [18]:
# group by interest_rate calculate rate of paid loans table clients has the interest_rate column and loans has the status column
query_interest_rate_paid = '''
SELECT
    c.interest_rate,
    COUNT(c.user_id) AS total_clients,
    SUM(CASE WHEN l.status = 'paid' THEN 1 ELSE 0 END) AS paid_loans,
    SUM(CASE WHEN l.status = 'paid' THEN 1.0 ELSE 0 END) / COUNT(c.user_id) AS rate_paid_loans
FROM
    clients c
LEFT JOIN loans l ON c.user_id = l.user_id
GROUP BY interest_rate;
'''

execute_query(query_interest_rate_paid)

,interest_rate,total_clients,paid_loans,rate_paid_loans
0,30,42214,22271,0.527574
1,90,42793,22579,0.527633
2,70,41963,22103,0.526726
3,20,42779,22642,0.529278


Overall, it seems that regardless of the interest rate, the rate of paid loans is quite similar, hovering around 52-53%. This suggests that, based on the provided data, there doesn't appear to be a significant difference in loan repayment rates based solely on the interest rate offered. However, further analysis, possibly including factors like loan amounts, client demographics, and economic conditions, would provide a more comprehensive understanding of the relationship between interest rates and loan repayment.

## 4. Rank the best 10 and 10 worst clients. Explain your methodology for constructing this ranking.

In [19]:
query_ranking = '''
WITH PaymentPerformance AS (
    SELECT
        c.user_id,
        COUNT(CASE WHEN l.status = 'paid' THEN 1 END) AS paid_loans,
        COUNT(l.loan_id) AS total_loans,
        COUNT(CASE WHEN l.status = 'paid' THEN 1 END) * 100.0 / NULLIF(COUNT(l.loan_id), 0) AS payment_rate
    FROM
        Clients c
    LEFT JOIN
        Loans l ON c.user_id = l.user_id
    WHERE
        c.status = 'approved'
    GROUP BY
        c.user_id
)
SELECT
    user_id,
    paid_loans,
    total_loans,
    payment_rate,
    ROW_NUMBER() OVER (ORDER BY payment_rate DESC) AS ranking
FROM
    PaymentPerformance
WHERE
    total_loans >= 5
ORDER BY
    payment_rate DESC;
'''


df = execute_query(query_ranking)
display(df.head(10))
display(df.tail(10))

,user_id,paid_loans,total_loans,payment_rate,ranking
0,84878,5,5,100.0,1
1,45007,5,5,100.0,2
2,55496,6,6,100.0,3
3,77401,5,5,100.0,4
4,49381,6,6,100.0,5
5,64827,5,5,100.0,6
6,65125,5,5,100.0,7
7,73264,5,5,100.0,8
8,50001,5,5,100.0,9
9,78645,5,5,100.0,10


,user_id,paid_loans,total_loans,payment_rate,ranking
2320,19499,4,5,80.0,2321
2321,38637,4,5,80.0,2322
2322,49405,4,5,80.0,2323
2323,11998,4,5,80.0,2324
2324,40702,4,5,80.0,2325
2325,23856,4,5,80.0,2326
2326,73871,4,5,80.0,2327
2327,45583,4,5,80.0,2328
2328,81486,4,5,80.0,2329
2329,36328,4,5,80.0,2330


the top 10 clients listed have demonstrated the highest payment rates, indicating a strong track record of loan repayment. Conversely, the bottom 10 clients listed have lower payment rates, indicating a lower rate of loan repayment compared to others.

## 5. What is the default rate by month and batch?

In [20]:
#default rate by month and batch
query_default_rate = '''
SELECT 
    month_batch_loan.batch,
    month_batch_loan.loan_month,
    month_batch_loan.total_loans AS total_loans_month_batch,
    month_batch_loan.defaulted_loans AS defaulted_loans_month_batch,
    ROUND(month_batch_loan.default_rate * 100, 2) AS default_rate_month_batch
FROM
    (SELECT 
        EXTRACT(MONTH FROM l.created_at) AS loan_month,
        c.batch,
        COUNT(*) AS total_loans,
        SUM(CASE WHEN l.status = 'default' THEN 1 ELSE 0 END) AS defaulted_loans,
        AVG(CASE WHEN l.status = 'default' THEN 1.0 ELSE 0 END) AS default_rate
    FROM Loans l
    JOIN Clients c ON l.user_id = c.user_id
    GROUP BY EXTRACT(MONTH FROM l.created_at), c.batch) AS month_batch_loan
ORDER BY default_rate_month_batch ASC;
'''

execute_query(query_default_rate)

,batch,loan_month,total_loans_month_batch,defaulted_loans_month_batch,default_rate_month_batch
0,4,12.0,925,21,2.27
1,4,1.0,837,20,2.39
2,3,11.0,1061,28,2.64
3,1,12.0,14819,394,2.66
4,4,11.0,773,22,2.85
5,1,1.0,14424,426,2.95
6,3,12.0,1357,40,2.95
7,3,1.0,1284,41,3.19
8,1,11.0,11729,375,3.20
9,2,12.0,5453,176,3.23


Batches and months with lower default rates are generally more favorable because they indicate a lower likelihood of loan defaults.

## 6. Assess the profitability of this operation. Provide an analysis of the operation's timeline.

Total Revenue

In [21]:
#total revenue
query_total_revenue = '''
SELECT SUM(loan_amount) AS total_revenue
FROM Loans
WHERE status = 'paid';

'''

execute_query(query_total_revenue)

,total_revenue
0,2.255315e+09


Total Expenses (including tax and interest)

(loan_amount * (interest_rate / 100) * 90 / 365)

- Tax: The tax is applied to the loan amount. In this scenario, the tax is calculated as 3.8% of the principal amount plus 0.0082% of the principal amount per day for 90 days.
- loan_amount: The principal amount borrowed.
- interest_rate: The annual interest rate assigned to the user.
- 90 / 365: This represents the portion of the year that the loan is active (90 days out of 365).

In [22]:
#Total Expenses
query_total_expenses = '''
SELECT SUM(loan_amount + tax + (loan_amount * (c.interest_rate / 100) * 90 / 365)) AS total_expenses
FROM Loans l
JOIN Clients c ON l.user_id = c.user_id
WHERE l.status = 'paid';

'''

execute_query(query_total_expenses)

,total_expenses
0,2.280529e+09


Total Profit

In [23]:
# Total Profit
query_total_profit = '''
SELECT (SELECT SUM(l.loan_amount) FROM Loans l JOIN Clients c ON l.user_id = c.user_id WHERE l.status = 'paid') -
       (SELECT SUM(l.loan_amount + l.tax + 90 * (l.loan_amount * (c.interest_rate / 100) / 365)) FROM Loans l JOIN Clients c ON l.user_id = c.user_id WHERE l.status = 'paid') AS total_profit;
'''

execute_query(query_total_profit)

,total_profit
0,-2.521442e+07


Profitability Analysis over Time

In [24]:
#Profitability Analysis over Time
query_profitability = """
SELECT DATE_TRUNC('month', l.created_at) AS month_year,
       SUM(l.loan_amount) AS total_revenue,
       SUM(l.loan_amount + l.tax + (l.loan_amount * (c.interest_rate / 100) * 90 / 365)) AS total_expenses,
       (SUM(l.loan_amount) - SUM(l.loan_amount + l.tax + (l.loan_amount * (c.interest_rate / 100) * 90 / 365))) AS total_profit
FROM Loans l
JOIN Clients c ON l.user_id = c.user_id
WHERE l.status = 'paid'
GROUP BY DATE_TRUNC('month', l.created_at)
ORDER BY month_year;

"""



execute_query(query_profitability)

,month_year,total_revenue,total_expenses,total_profit
0,2020-01-01,288922.0,2.921521e+05,-3.230150e+03
1,2020-02-01,1353744.0,1.368879e+06,-1.513487e+04
2,2020-03-01,2275957.0,2.301402e+06,-2.544519e+04
3,2020-04-01,2987108.0,3.020504e+06,-3.339586e+04
4,2020-05-01,3543760.0,3.583379e+06,-3.961924e+04
5,2020-06-01,5172997.0,5.230831e+06,-5.783414e+04
6,2020-07-01,6114630.0,6.182992e+06,-6.836159e+04
7,2020-08-01,7099692.0,7.179067e+06,-7.937451e+04
8,2020-09-01,7878028.0,7.966104e+06,-8.807633e+04
9,2020-10-01,9665109.0,9.773165e+06,-1.080560e+05
